Connected to Python 3.11.4

In [1]:
from bs4 import BeautifulSoup, element
import urllib
import urllib.request
import pandas as pd
import numpy as np
import os

In [19]:
pages = 2
rec_count = 0
rank = []
gname = []
platform = []
year = []
genre = []
critic_score = []
user_score = []
publisher = []
developer = []
sales_na = []
sales_pal = []
sales_jp = []
sales_ot = []
sales_gl = []
# new list to hold href url_to_game
ind_game_url = []
#altered_ind_game_url = []
# new list(s) to hold supplementary sales data according to the new plan
#ind_sales_na = []
#ind_sales_pal = []
#ind_sales_jp = []
#ind_sales_ot = []
#ind_sales_gl = []


# updated url tails to account for order, order = TotalSales
# deselecting VGChartz Score
urlhead = 'http://www.vgchartz.com/gamedb/?page='
urltail = '&console=&region=All&developer=&publisher=&genre=&boxart=Both&ownership=Both'
# setting to 10 for test, otherwise 1000
urltail += '&results=10&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0'
urltail += '&showpublisher=1&showvgchartzscore=0&shownasales=1&showdeveloper=1&showcriticscore=1'
urltail += '&showpalsales=0&showpalsales=1&showreleasedate=1&showuserscore=1&showjapansales=1'
urltail += '&showlastupdate=0&showothersales=1&showgenre=1&sort=GL'



for page in range(1, pages):
    surl = urlhead + str(page) + urltail
    # urllib error, no att request
    r = urllib.request.urlopen(surl, data = None).read()
    # add features = html.parser
    soup = BeautifulSoup(r,features = "html.parser" )
    print(f"Page: {page}")

    # page is being scanned for any links that contain the game link
    # <a> tags with game urls
    # swapped to https
    game_tags = list(filter(
        lambda x: 'href' in x.attrs and x.attrs['href'].startswith('https://www.vgchartz.com/game/'),
        # discard the first 10 elements because those
        # links are in the navigation bar
        soup.find_all("a")
    ))[1:]

    for tag in game_tags:
        
        # add name to list
        gname.append(" ".join(tag.string.split()))
        print(f"{rec_count + 1} Fetch data for game {gname[-1]}")

        # get different attributes
        # traverse up the DOM tree
        data = tag.parent.parent.find_all("td")
        rank.append(np.int32(data[0].string))
        platform.append(data[3].find('img').attrs['alt'])
        publisher.append(data[4].string)
        developer.append(data[5].string)
        critic_score.append(
            float(data[6].string) if
            not data[6].string.startswith("N/A") else np.nan)
        user_score.append(
            float(data[7].string) if
            not data[7].string.startswith("N/A") else np.nan)
        # most of the sales data that was indicated on the website is now populating as N/A
        # I think it's mostly an issue with how the main GameDB table is populating, reason being
        # if you click on each individual game url and navigate to the sales tab, the sales data is there, it exists
        # so, new plan:
        # in the same vein of when we grab the genre info, by detecting and reading each individual game url
        # we ALSO take those urls, chop of their tails, add the word sales and form a new GET request
        sales_na.append(
            float(data[9].string[:-1]) if
            not data[9].string.startswith("N/A") else np.nan)
        sales_pal.append(
            float(data[10].string[:-1]) if
            not data[10].string.startswith("N/A") else np.nan)
        sales_jp.append(
            float(data[11].string[:-1]) if
            not data[11].string.startswith("N/A") else np.nan)
        sales_ot.append(
            float(data[12].string[:-1]) if
            not data[12].string.startswith("N/A") else np.nan)
        sales_gl.append(
            float(data[8].string[:-1]) if
            not data[8].string.startswith("N/A") else np.nan)
        release_year = data[13].string.split()[-1]
        # different format for year
        if release_year.startswith('N/A'):
            year.append('N/A')
        else:
            if int(release_year) >= 80:
                year_to_add = np.int32("19" + release_year)
            else:
                year_to_add = np.int32("20" + release_year)
            year.append(year_to_add)

        # go to every individual website to get genre info
        url_to_game = tag.attrs['href']
        site_raw = urllib.request.urlopen(url_to_game).read()
        sub_soup = BeautifulSoup(site_raw, "html.parser")
        # again, the info box is inconsistent among games so we
        # have to find all the h2 and traverse from that to the genre name
        h2s = sub_soup.find("div", {"id": "gameGenInfoBox"}).find_all('h2')
        # make a temporary tag here to search for the one that contains
        # the word "Genre"
        temp_tag = element.Tag
        for h2 in h2s:
            if h2.string == 'Genre':
                temp_tag = h2
        genre.append(temp_tag.next_sibling.string)

        # new plan taketh shape
        # individual game url column
        ind_game_url.append(url_to_game)
        # new tail piece that will take us to the sales page of each game
       # new_tail = "sales"
       # last_slash_index = ind_game_url.rfind('/')
       # for tails


        rec_count += 1

columns = {
    'Rank': rank,
    'Name': gname,
    'Platform': platform,
    'Year': year,
    'Genre': genre,
    'Critic_Score': critic_score,
    'User_Score': user_score,
    'Publisher': publisher,
    'Developer': developer,
    'NA_Sales': sales_na,
    'PAL_Sales': sales_pal,
    'JP_Sales': sales_jp,
    'Other_Sales': sales_ot,
    'Global_Sales': sales_gl,
    'Game_URL': ind_game_url
}
   #'Ind_NA_Sales': ind_sales_na,
    #'Ind_PAL_Sales': ind_sales_pal,
    #'Ind_JP_Sales': ind_sales_jp,
    #'Ind_Other_Sales': ind_sales_ot,
    #'Ind_Global_Sales': ind_sales_gl

Page: 1
1 Fetch data for game Pokemon
2 Fetch data for game Super Mario
3 Fetch data for game Call of Duty
4 Fetch data for game Grand Theft Auto
5 Fetch data for game FIFA
6 Fetch data for game Minecraft
7 Fetch data for game Minecraft
8 Fetch data for game Lego
9 Fetch data for game Assassin's Creed


In [24]:
print(rec_count)
df = pd.DataFrame(columns)
print(df.columns)
df = df[[
    'Rank', 'Name', 'Platform', 'Year', 'Genre',
    'Publisher', 'Developer', 'Critic_Score', 'User_Score',
    'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales',
    'Game_URL']]

#'Ind_NA_Sales', 'Ind_PAL_Sales', 'Ind_JP_Sales', 'Ind_Other_Sales', 'Ind_Global_Sales'

9
Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Critic_Score',
       'User_Score', 'Publisher', 'Developer', 'NA_Sales', 'PAL_Sales',
       'JP_Sales', 'Other_Sales', 'Global_Sales', 'Game_URL'],
      dtype='object')


In [25]:
# change the name of the file
df.to_csv(os.getcwd()+r"\vgsales_xx.csv", sep=",", encoding='utf-8', index=False)